# Scenario Analysis: Policy-Conditional Forecasting

This notebook demonstrates the scenario analysis framework using MS-VARX conditional forecasting.

**Contents:**
1. Load scenario definitions and results
2. Visualize scenario fan chart
3. Compare adverse vs upside scenarios
4. Sensitivity analysis
5. Policy implications

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 11

# Scenario colors
SCENARIO_COLORS = {
    'Baseline': '#2E86AB',
    'Combined Upside': '#28A745',
    'Tourism Recovery (+25%)': '#20C997',
    'Combined Adverse': '#DC3545',
    'LKR Depreciation 20%': '#E83E8C',
    'Export Shock (-15%)': '#FD7E14',
    'Remittance Decline (-20%)': '#6F42C1',
    'Oil Price Shock': '#17A2B8',
    'IMF Tranche Delay': '#6C757D',
    'LKR Depreciation 10%': '#FFC107'
}

## 1. Scenario Definitions

We define 10 policy-relevant scenarios with specific shock multipliers.

In [ ]:
# Scenario definitions
scenarios = {
    'Baseline': {
        'description': 'Current trajectory continues',
        'shocks': {}
    },
    'LKR Depreciation 10%': {
        'description': 'Moderate exchange rate pressure',
        'shocks': {'usd_lkr': 1.10}
    },
    'LKR Depreciation 20%': {
        'description': 'Severe currency crisis',
        'shocks': {'usd_lkr': 1.20}
    },
    'Export Shock (-15%)': {
        'description': 'Global demand contraction',
        'shocks': {'exports_usd_m': 0.85, 'trade_balance_usd_m': 0.70}
    },
    'Remittance Decline (-20%)': {
        'description': 'Gulf employment crisis',
        'shocks': {'remittances_usd_m': 0.80}
    },
    'Tourism Recovery (+25%)': {
        'description': 'Post-COVID tourism boom',
        'shocks': {'tourism_usd_m': 1.25}
    },
    'Oil Price Shock': {
        'description': 'Energy import surge',
        'shocks': {'imports_usd_m': 1.15, 'trade_balance_usd_m': 0.75}
    },
    'IMF Tranche Delay': {
        'description': 'Disbursement postponed 6 months',
        'shocks': {'usd_lkr': 1.08, 'imports_usd_m': 0.95}
    },
    'Combined Adverse': {
        'description': 'Multiple simultaneous shocks (stress test)',
        'shocks': {
            'exports_usd_m': 0.90,
            'remittances_usd_m': 0.85,
            'tourism_usd_m': 0.80,
            'imports_usd_m': 1.10,
            'usd_lkr': 1.15
        }
    },
    'Combined Upside': {
        'description': 'Favorable conditions across the board',
        'shocks': {
            'exports_usd_m': 1.10,
            'remittances_usd_m': 1.10,
            'tourism_usd_m': 1.20,
            'imports_usd_m': 0.95,
            'usd_lkr': 0.95
        }
    }
}

# Display as table
scenario_df = pd.DataFrame([
    {'Scenario': k, 'Description': v['description'], 'Key Shocks': str(v['shocks']) if v['shocks'] else 'None'}
    for k, v in scenarios.items()
])
print(scenario_df.to_string(index=False))

## 2. Load Scenario Results

Load pre-computed MS-VAR scenario forecasts.

In [ ]:
# Load scenario results
try:
    results = pd.read_csv('../data/scenario_analysis/msvar_scenario_results.csv')
    paths = pd.read_csv('../data/scenario_analysis/msvar_scenario_paths.csv')
    
    print("Loaded scenario results!")
    print(f"\nScenarios: {results['Scenario'].tolist()}")
except FileNotFoundError:
    print("Scenario results not found.")
    print("Run 'slreserves scenarios' to generate results.")
    results = None
    paths = None

In [ ]:
if results is not None:
    # Display summary table
    summary = results[['Scenario', 'Description', 'Start (USD M)', 'End (USD M)', 'Change (%)']].\
        rename(columns={'Start (USD M)': 'Start', 'End (USD M)': 'End', 'Change (%)': 'Change %'})
    summary['Change %'] = summary['Change %'].round(1)
    summary['Start'] = summary['Start'].round(0).astype(int)
    summary['End'] = summary['End'].round(0).astype(int)
    
    print("=== 12-Month Scenario Forecasts ===")
    print(summary.to_string(index=False))

## 3. Scenario Fan Chart

Visualize all scenario paths together.

In [ ]:
if paths is not None:
    # Load historical data for context
    hist = pd.read_csv('../data/merged/reserves_forecasting_panel.csv', 
                       parse_dates=['date'], index_col='date')
    hist_recent = hist['gross_reserves_usd_m'].iloc[-24:]  # Last 24 months
    
    fig, ax = plt.subplots(figsize=(14, 8))
    
    # Plot historical
    ax.plot(hist_recent.index, hist_recent.values, 
            linewidth=2.5, color='black', label='Historical')
    
    # Plot scenarios
    for scenario in results['Scenario']:
        scenario_data = paths[paths['Scenario'] == scenario] if 'Scenario' in paths.columns else \
                       paths[[scenario]].iloc[:, 0]
        
        # Handle different data formats
        if 'Scenario' in paths.columns:
            # Long format
            vals = paths[paths['Scenario'] == scenario]['gross_reserves_usd_m'].values
            dates_str = paths[paths['Scenario'] == scenario]['date'].values
        else:
            # Wide format
            vals = paths[scenario].values
            dates_str = paths.index if hasattr(paths.index, 'strftime') else paths.iloc[:, 0].values
        
        # Create date index
        forecast_start = hist.index[-1] + pd.DateOffset(months=1)
        dates = pd.date_range(start=forecast_start, periods=len(vals), freq='MS')
        
        color = SCENARIO_COLORS.get(scenario, '#999999')
        linewidth = 2.5 if scenario in ['Baseline', 'Combined Adverse', 'Combined Upside'] else 1.5
        linestyle = '-' if scenario == 'Baseline' else '--'
        
        ax.plot(dates, vals, linewidth=linewidth, linestyle=linestyle,
                color=color, label=scenario, alpha=0.9)
    
    # Formatting
    ax.axvline(hist.index[-1], color='gray', linestyle=':', alpha=0.7)
    ax.text(hist.index[-1], ax.get_ylim()[1]*0.98, ' Forecast →', 
            va='top', fontsize=10, color='gray')
    
    ax.set_xlabel('Date')
    ax.set_ylabel('Gross Reserves (USD millions)')
    ax.set_title('MS-VAR Scenario Fan Chart: 12-Month Horizon')
    ax.legend(loc='center left', bbox_to_anchor=(1.02, 0.5), fontsize=9)
    
    plt.tight_layout()
    plt.show()

## 4. Scenario Comparison: Bar Chart

In [ ]:
if results is not None:
    fig, axes = plt.subplots(1, 2, figsize=(14, 6))
    
    # Sort by end level
    sorted_results = results.sort_values('End (USD M)', ascending=True)
    
    # End level comparison
    colors = [SCENARIO_COLORS.get(s, '#999999') for s in sorted_results['Scenario']]
    axes[0].barh(sorted_results['Scenario'], sorted_results['End (USD M)'], color=colors)
    axes[0].set_xlabel('End Level (USD M)')
    axes[0].set_title('Projected Reserves After 12 Months')
    
    # Add baseline reference
    baseline_end = results[results['Scenario'] == 'Baseline']['End (USD M)'].values[0]
    axes[0].axvline(baseline_end, color='black', linestyle='--', alpha=0.7, label='Baseline')
    
    # Change percentage
    sorted_by_change = results.sort_values('Change (%)', ascending=True)
    colors = [SCENARIO_COLORS.get(s, '#999999') for s in sorted_by_change['Scenario']]
    axes[1].barh(sorted_by_change['Scenario'], sorted_by_change['Change (%)'], color=colors)
    axes[1].set_xlabel('Change (%)')
    axes[1].set_title('Percentage Change from Current Level')
    axes[1].axvline(0, color='black', linestyle='-', alpha=0.5)
    
    plt.tight_layout()
    plt.show()

## 5. Stress Test Range

In [ ]:
if results is not None:
    # Calculate range statistics
    start = results['Start (USD M)'].iloc[0]
    best = results['End (USD M)'].max()
    worst = results['End (USD M)'].min()
    baseline = results[results['Scenario'] == 'Baseline']['End (USD M)'].values[0]
    
    print("="*60)
    print("STRESS TEST SUMMARY")
    print("="*60)
    print(f"""
Current Level:     ${start:,.0f} M

12-Month Projections:
  Best Case:       ${best:,.0f} M  (+{(best/start-1)*100:.1f}%)  [Combined Upside]
  Baseline:        ${baseline:,.0f} M  (+{(baseline/start-1)*100:.1f}%)
  Worst Case:      ${worst:,.0f} M  (+{(worst/start-1)*100:.1f}%)  [Combined Adverse]

Range Width:       ${best - worst:,.0f} M
Downside Risk:     ${baseline - worst:,.0f} M below baseline
Upside Potential:  ${best - baseline:,.0f} M above baseline
""")

In [ ]:
if results is not None:
    # Waterfall-style visualization
    fig, ax = plt.subplots(figsize=(10, 8))
    
    # Create range visualization
    ax.fill_betweenx([0, 1], start, best, alpha=0.3, color='green', label='Upside range')
    ax.fill_betweenx([0, 1], worst, start, alpha=0.3, color='red', label='Downside range')
    
    # Key levels
    levels = [
        (worst, 'Combined Adverse', '#DC3545'),
        (start, 'Current', '#2E86AB'),
        (baseline, 'Baseline', 'black'),
        (best, 'Combined Upside', '#28A745')
    ]
    
    for val, label, color in levels:
        ax.axvline(val, color=color, linewidth=2, linestyle='-' if label in ['Current', 'Baseline'] else '--')
        ax.text(val, 1.05, f'{label}\n${val/1000:.1f}B', ha='center', fontsize=10, color=color)
    
    ax.set_xlim(worst * 0.95, best * 1.05)
    ax.set_ylim(0, 1.2)
    ax.set_xlabel('Gross Reserves (USD millions)')
    ax.set_title('12-Month Reserve Projection Range')
    ax.set_yticks([])
    ax.legend(loc='lower right')
    
    plt.tight_layout()
    plt.show()

## 6. Policy Implications

In [ ]:
print("""
================================================================================
POLICY IMPLICATIONS
================================================================================

1. TOURISM IS THE LARGEST SINGLE DRIVER
   - Tourism Recovery (+25%) adds +$1.2B to reserves over 12 months
   - Priority: Tourism promotion, infrastructure, visa facilitation
   
2. COMBINED SHOCKS ARE SEVERE BUT MANAGEABLE
   - Combined Adverse scenario still shows slight reserve increase (+1%)
   - Current trajectory provides buffer against individual shocks
   - IMF program backstop limits downside

3. EXCHANGE RATE SENSITIVITY
   - 20% LKR depreciation: relatively limited reserve impact
   - Suggests CBSL can allow gradual FX adjustment
   - Avoid large interventions that drain reserves

4. REMITTANCE DEPENDENCE
   - Gulf employment remains critical
   - Diversify labor markets (Japan, Korea, Eastern Europe)
   - Formalize hawala channels to capture flows

5. IMF PROGRAM CRITICALITY
   - Tranche delay has moderate impact (+2.7% instead of baseline)
   - Maintain program compliance to prevent larger market reaction
================================================================================
""")

## Run Your Own Scenarios

Use the command line to run custom scenarios:

```bash
# Run all scenarios
slreserves scenarios --scenarios all

# Run specific scenarios
slreserves scenarios --scenarios combined_adverse,tourism_recovery

# Custom horizon
slreserves scenarios --horizon 24 --scenarios baseline
```